In [ ]:
import torch
from torch.utils.data import Dataset

In [4]:
camembert = torch.hub.load('pytorch/fairseq', 'camembert.v0')

Using cache found in /home/andres/.cache/torch/hub/pytorch_fairseq_master


In [5]:
class FactsOrAnalysisDS_BERT(Dataset):
    """PyTorch Dataset class. Returns input-target tensor pairs"""
    def __init__(self,csv_file):
        camembert = torch.hub.load('pytorch/fairseq', 'camembert.v0')
        with open(csv_file) as f:
            dataset=[line.split(",") for line in f]
        self.dataset=[[camembert.encode(datum[0]),int(datum[1][0])] for datum in dataset]
    
    def __len__(self):
        return (len(self.dataset))
    
    def __getitem__(self,index):
        return self.dataset[index]

In [6]:
data_file="data/train_data.csv"
with open(data_file) as f:
    dataset=[line.split(",") for line in f]
dataset=[[pair[0].replace("\n",""),pair[1].replace("\n","")] for pair in dataset]

In [9]:
dataset_camembert=FactsOrAnalysisDS_BERT(data_file)

Using cache found in /home/andres/.cache/torch/hub/pytorch_fairseq_master


In [25]:
import torch
rnn=torch.nn.GRU(
    input_size=768,
    hidden_size=64,
    num_layers=3,
    batch_first=True,
    bidirectional=True)

In [15]:
u=dataset_camembert[90][0].unsqueeze(0)
# u=u.unsqueeze(0)
# u=camembert.encode(u)
# u=camembert.extract_features(u)
# u,v=rnn(u)
# v.shape
u

tensor([[    5,    22,    44,    31,  1307,    16, 11938, 19299,    16,    17,
          1920,  1569,  9046,    27,    16,  9813,   111,   279,    13,  2124,
             8,   191,    16, 11938,     8,   742,    15,   652,     6]])

In [75]:
camembert.train()
camembert.extract_features(u)

tensor([[[ 0.0574,  0.1625,  0.1480,  ..., -0.0822, -0.0142,  0.0169],
         [-0.0104,  0.3305,  0.2646,  ..., -0.1891,  0.0838, -0.0138],
         [ 0.0456,  0.2597, -0.0835,  ...,  0.0257,  0.1584,  0.0653],
         ...,
         [ 0.1277,  0.1146,  0.0857,  ...,  0.0499,  0.0650, -0.1046],
         [-0.0506, -0.3077, -0.1230,  ..., -0.0575,  0.0098,  0.2734],
         [-0.0287,  0.1020,  0.1440,  ..., -0.0635,  0.0814,  0.1171]]],
       grad_fn=<TransposeBackward0>)

I have to rewrite the `Dataset` class to accomodate camemBERT

In [125]:
import torch

13

In [76]:
class camembertTest(torch.nn.Module):
    def __init__(self,cam):
        super(camembertTest, self).__init__()
        self.camembert=cam
#         self.camembert.eval()
        
    def forward(self,seq):
        return self.camembert.extract_features(seq)

In [77]:
model=camembertTest(camembert)

In [86]:
model(u)

tensor([[[ 0.0633,  0.0881,  0.0851,  ...,  0.0363, -0.0519,  0.0749],
         [-0.0524,  0.3216,  0.2259,  ..., -0.1883,  0.0476,  0.0619],
         [ 0.0397,  0.1794, -0.1710,  ...,  0.0063,  0.0882,  0.0660],
         ...,
         [ 0.0571,  0.1803, -0.0355,  ..., -0.0590,  0.0469, -0.1190],
         [-0.0356, -0.2203, -0.1231,  ...,  0.0677,  0.0151,  0.2744],
         [-0.0027,  0.1280,  0.1507,  ..., -0.0694, -0.0657,  0.0486]]],
       grad_fn=<TransposeBackward0>)

In [101]:
model.train()

camembertTest(
  (camembert): RobertaHubInterface(
    (model): RobertaModel(
      (decoder): RobertaEncoder(
        (sentence_encoder): TransformerSentenceEncoder(
          (embed_tokens): Embedding(32005, 768, padding_idx=1)
          (embed_positions): LearnedPositionalEmbedding(514, 768, padding_idx=1)
          (layers): ModuleList(
            (0): TransformerSentenceEncoderLayer(
              (self_attn): MultiheadAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=True)
                (v_proj): Linear(in_features=768, out_features=768, bias=True)
                (q_proj): Linear(in_features=768, out_features=768, bias=True)
                (out_proj): Linear(in_features=768, out_features=768, bias=True)
              )
              (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
              (fc1): Linear(in_features=768, out_features=3072, bias=True)
              (fc2): Linear(in_features=3072, out_features=

In [105]:
model(u)

tensor([[[ 0.0557,  0.0921, -0.0298,  ..., -0.0734,  0.0880,  0.0880],
         [-0.0278,  0.2673,  0.1886,  ..., -0.1658,  0.0726, -0.0252],
         [-0.0204,  0.2479, -0.3003,  ...,  0.0726,  0.1601,  0.1031],
         ...,
         [ 0.1874, -0.0342, -0.0286,  ..., -0.0705,  0.0729, -0.1227],
         [-0.1576, -0.2899, -0.1430,  ...,  0.0161, -0.0232,  0.3373],
         [ 0.0415,  0.1347,  0.1356,  ..., -0.0966, -0.0670,  0.1503]]],
       grad_fn=<TransposeBackward0>)